In [11]:
import sklearn                                                # for neural networks and performance metrics
import numpy as np                                            # for math and data operations
import pandas as pd                                           # for dataframes used in documentation and debugging
import random                                                 # for use in training data creation
import math
import string
from threading import Thread
import time
from pyeasyga import pyeasyga
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [12]:
training_input_list = np.zeros((1000, 2))                           # creating a column vector for inputs. Determines training size of the rest of the data
for i in range(training_input_list.shape[0]):                        # populating the input column vector via a for loop
    training_input_list[i][0] = random.randint(0, 1)
    training_input_list[i][1] = random.randint(0, 1)
training_xor_list = np.zeros((training_input_list.shape[0],1))       # creating a column vector of sins for outputs
for i in range(training_xor_list.shape[0]):                          # calculating the appropriate outputs for the input column vector
    training_xor_list[i][0] = training_input_list[i][0] != training_input_list[i][1]      # as per the XOR logical operator

    
print(training_input_list[:10])           # this is just debug code to verify the shapes of the column vectors
print(training_xor_list[:10])

[[0. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 0.]
 [0. 1.]
 [0. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]]
[[0.]
 [1.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]]


In [13]:
trainingData = [training_input_list, training_xor_list]

In [14]:
#Defining a timed fitting function for neural networks to make sure candidate solutions are reasonably efficient
def timedFit(ANN, data):
    score = 0
    X_train, X_test, Y_train, Y_test = train_test_split(data[0], data[1], test_size=0.4)
    fitting = Thread(target=ANN.fit(), args=(X_train, Y_train))
    threading.join(fitting, timeout=5)
    score = ANN.score(X_test, Y_test)
    return score

In [15]:
#Instantiating the genetic algorithms
genetics = pyeasyga.GeneticAlgorithm(trainingData,
                               population_size=100,
                               generations=15,
                               crossover_probability=0.80,
                               mutation_probability=0.15,
                               elitism=True,
                               maximise_fitness=True)

In [20]:
#Defining the chromosomes' structure
def create_individual(data):                               # Each genome is a half byte representing the size of a hidden layer,
    individual = [random.randint(0, 1) for _ in range(20)]   # with each neural network having a maximum of 5 hidden layers
    for i in range(len(individual)):
        if not((i+1) % 4):
            individual[i] = int(random.randint(0, int(i/4)) != 0) if not(individual[i]) else individual[i]
    return individual
# create_individual function overwrites the default to minimize the odds that bits starting genomes of hidden layers will be 0,
# thereby reducing the odds of missing hidden layers in chromosomes of the initial population with minimal increase of bias to
# deeper networks. Odds of a genome-starting bit being replaced with a 1 are highest for layers closest to the start of the
# network, as the rest of the network construction throws out all layers right of a missing hidden layer.

In [21]:
def evaluate_genome(genome):     # Iterates through the list of bits of a genome and evaluates them as a binary number
    value = "0B"
    for bit in genome:
        value += str(bit)
    return int(value, 2)

In [22]:
#Defining the fitness function
def fitness(individual, data): #builds a neural network according to the genomes representing the architecture and tests
    fitness = 0                # the neural network
    genomes = [evaluate_genome(individual[0:4]),
               evaluate_genome(individual[4:8]),
               evaluate_genome(individual[8:12]),
               evaluate_genome(individual[12:16]),
               evaluate_genome(individual[16:20])]
    
    if genomes[0]:    # a series of checks to confirm that the shape represented by the genome is valid
        if genomes[1]: # will only construct a neural network using valid genomes from the left end of the chromosome
            if genomes[2]:
                if genomes[3]:
                    if genomes[4]:
                        neuralNetwork = MLPClassifier(hidden_layer_sizes=(genomes[0], genomes[1], genomes[2], genomes[3], genomes[4]), activation='tanh', solver='lbfgs', max_iter=100)
                    else:
                        neuralNetwork = MLPClassifier(hidden_layer_sizes=(genomes[0], genomes[1], genomes[2], genomes[3]), activation='tanh', solver='lbfgs', max_iter=100)
                else:
                    neuralNetwork = MLPClassifier(hidden_layer_sizes=(genomes[0], genomes[1], genomes[2]), activation='tanh', solver='lbfgs', max_iter=100)
            else:
                neuralNetwork = MLPClassifier(hidden_layer_sizes=(genomes[0], genomes[1]), activation='tanh', solver='lbfgs', max_iter=100)
        else:
            neuralNetwork = MLPClassifier(hidden_layer_sizes=(genomes[0]), activation='tanh', solver='lbfgs', max_iter=100)
    else:
        return fitness
    X_train, X_test, Y_train, Y_test = train_test_split(data[0], data[1], test_size=0.4)
    neuralNetwork.fit(X_train, np.ravel(Y_train))
    return neuralNetwork.score(X_test, np.ravel(Y_test))

In [23]:
genetics.create_individual = create_individual
genetics.fitness_function = fitness
genetics.run()

In [24]:
print(genetics.best_individual())

(1.0, [1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1])
